In [1]:
import pandas as pd

In [2]:
def get_houseNo(file_path):
    # 先提取鸡舍号信息
    df_Hn = pd.read_excel(file_path, sheet_name='基本信息', header=None)
    header_index = 5  # 假设表头在第 6 行（Excel 行号），对应 pandas 索引 5
    header = df_Hn.iloc[header_index]
    house_no_col = None
    for col in header.index:
        if header[col] == '鸡舍号\nHouse No':
            house_no_col = col
            break

    if house_no_col is None:
        print("未找到鸡舍号列")
    else:
        # 提取该列数据（从第 7 行开始，Excel 行号，对应 pandas 索引 6），填充合并单元格导致的 NaN
        house_nos = df_Hn.iloc[6:, house_no_col].dropna(axis=0).tolist()
        house_nos=[x for x in house_nos if x.startswith('H')]
        print("鸡舍号信息：", house_nos)
    return house_nos

In [3]:
def get_chicken_house_info(df,house_nos):
    n_house=len(house_nos)
    n_cols=n_house*4

    chicken_house_info = df.iloc[1:4].copy()
    chicken_house_info=chicken_house_info.dropna(axis=1)
    chicken_house_info=chicken_house_info.iloc[:,:n_cols]

    result = []
    for i in range(0, chicken_house_info.shape[1], 4):
        group = chicken_house_info.iloc[:, i:i+4]

        # 提取对应位置的值
        house_no = group.iloc[0, 1]
        gender = group.iloc[0, 3]
        birds_placed = group.iloc[1, 1]
        breeder = group.iloc[1, 3]
        doc_date = group.iloc[2, 1]
        age = group.iloc[2, 3]
        result.append([house_no, gender, birds_placed, breeder, doc_date, age])

    new_chicken_house_info = pd.DataFrame(
        result, 
        columns=['House No.', 'Gender', 'Birds placed', 'Breeder', 'DOC Date', 'Age']
    )
    # print(new_chicken_house_info)
    return new_chicken_house_info

In [4]:
def get_dead_data(df,house_nos):
    # 提取死淘数据（假设从第6行开始，索引5）
    n_house=len(house_nos)
    n_cols=n_house*10+1

    dead_data = df.iloc[5:,:n_cols]
    # 设置死淘数据列名（假设第6行为列名，索引5）
    dead_data.columns = dead_data.iloc[0]
    dead_data = dead_data[1:]
    dead_data=dead_data.reset_index(drop=True)


    # 查找“合计”所在行的索引
    drop_index = None
    for index, row in dead_data.iterrows():
        if row['Date'] == '合计':
            drop_index = index
            break

    # 如果找到“合计”行，则删除该行及之后的所有行
    if drop_index is not None:
        dead_data = dead_data.iloc[:drop_index]


    # 将第 1 行（索引 0）设置为列名
    dead_data.columns = dead_data.iloc[0]
    # 删除第 2 行（索引 1）
    dead_data = dead_data.iloc[2:]




    result = []
    for i, house in enumerate(house_nos):
        start_col = 1 + i * 10
        end_col = start_col + 10
        # 提取日期和对应鸡舍的 10 列数据
        row_data = dead_data.iloc[:,[0]]
        row_data = row_data.join(dead_data.iloc[:, start_col:end_col])
        row_data['House_No'] = house
        # print(row_data)
        result.append(row_data)

    new_dead_data = pd.concat(result, ignore_index=True)
    new_dead_data.columns=['Date',
    'Age',
    'Dead',
    'Swollen_Head',
    'Weak',
    'Navel_Disease',
    'Stick_Anus',
    'Lame_Paralysis',
    'Mortality',
    'Mortality_rate',
    'Remark',
    'House_No']
    return new_dead_data

In [11]:
chicken_house_info

,House No.,Gender,Birds placed,Breeder,DOC Date,Age,id_no
0,H1,M&F,25500,L3,2024-11-20,26,01A_62
1,H2,M&F,25700,L4,2024-11-20,45,01A_62
2,H3,M&F,25700,L4,2024-11-20,45,01A_62
3,H4,M&F,25900,L4,2024-11-20,45,01A_62
4,H5,M&F,25500,L5,2024-11-20,51,01A_62


In [ ]:
# 单文件测试
file_path = 'C:\\FILES_202305\\202504养鸡项目\\chickenfarming\\data\\24.12 2\\日报\\01A_62.xlsm'  # 将此处替换为实际的文件路径
id_no='01A_62'
df = pd.read_excel(file_path, sheet_name='死淘分类',skiprows=5)
house_nos=get_houseNo(file_path)
df = pd.read_excel(file_path, sheet_name='死淘分类', header=None)
chicken_house_info=get_chicken_house_info(df,house_nos)
chicken_house_info['id_no']=id_no
dead_data=get_dead_data(df,house_nos)
dead_data['id_no']=id_no

鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20']


In [ ]:
import os
import pandas as pd



# 文件夹路径
folder_path = 'C:\\FILES_202305\\202504养鸡项目\\chickenfarming\\data\\all_daily_data'  # 请替换为实际的文件夹路径

# 存储所有结果的列表
all_chicken_house_info =pd.DataFrame()
all_dead_data=pd.DataFrame()

# 遍历文件夹中的所有文件
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xlsm'):
            id_no = os.path.splitext(file)[0]
            file_path = os.path.join(root, file)
            print(id_no)
            # 调用处理函数
            house_nos=get_houseNo(file_path)
            df = pd.read_excel(file_path, sheet_name='死淘分类', header=None)
            chicken_house_info=get_chicken_house_info(df,house_nos)
            chicken_house_info['id_no']=id_no
            dead_data=get_dead_data(df,house_nos)
            dead_data['id_no']=id_no

            all_chicken_house_info=pd.concat([all_chicken_house_info,chicken_house_info])
            all_dead_data=pd.concat([all_dead_data,dead_data])

# 将所有结果合并到一个新的DataFrame中
# merged_df = pd.concat(all_results, ignore_index=True)

# print(merged_df)

00_71
鸡舍号信息： ['H1', 'H2', 'H3']
00_73
鸡舍号信息： ['H1', 'H2', 'H3']
01A_62
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20']
01A_63
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20']
02_61
鸡舍号信息： ['H1', 'H2', 'H8', 'H9', 'H10']
02_62
鸡舍号信息： ['H1', 'H2', 'H8', 'H9', 'H10']
03_66
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10']
03_67
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10']
04_50
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8']
04_51
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10']
06_65
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20']
06_66
鸡舍号信息： ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17'

In [21]:
all_dead_data.to_csv('C:\\FILES_202305\\202504养鸡项目\\chickenfarming\\data\\data_cleaned\\all_dead_data.csv',index=False)

In [22]:
all_chicken_house_info.to_csv('C:\\FILES_202305\\202504养鸡项目\\chickenfarming\\data\\data_cleaned\\all_chicken_house_info.csv',index=False)
